In [121]:
# Step 1: Import necessary libraries
from sklearn.cluster import AgglomerativeClustering
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage
from utils.classification_data import linear_data, checkerboard_data, power_line_data, microgrid_data, make_double_cake_data
from sklearn.model_selection import train_test_split
from pennylane import numpy as np
import pennylane as qml
from pennylane import numpy as np
import matplotlib as mpl
from pennylane import numpy as np
from sklearn.cluster import SpectralClustering, KMeans
from collections import deque
import pandas as pd
from utils.classification_data import plot_and_save, make_double_cake_data

In [122]:
dataset = 'iris'
n_clusters = 4

In [123]:
features, target = make_double_cake_data(3, 10)

In [124]:
#data = pd.read_csv('data/balanced_quantum_dataset.csv')
#features = np.asarray(data.drop(columns=['label']))
#target = np.asarray(data['label'])

X, x_test, Y, y_test = train_test_split(
    features, target, test_size=0.8, random_state=42
)

In [125]:
def train_test_split(data_filepath, num_train=10, num_test=10, balanced=False):
    """
    Quick function used to split data into test and training samples.

    Args:
        data_filepath (str): filepath to read the data from.
        num_train (int): number of training samples.
        num_test (int): number of test samples.
        balanced (bool): whether there are an equal number of each class represented.

    Returns:
        X_train (np.ndarray): training sample features.
        y_train (np.ndarray): testing sample features.
        X_test (np.ndarray): training sample labels.
        y_test (np.ndarray): testing sample labels.
    """
    df = pd.read_csv(data_filepath, sep=",")
    train_indices = []
    test_indices = []

    if balanced:
        unique_labels = df.iloc[:, -1].unique()

        for label in unique_labels:
            label_indices = df[df.iloc[:, -1] == label].index.to_numpy()
            np.random.shuffle(label_indices)

            train_count = min(num_train, len(label_indices))
            test_count = min(num_test, len(label_indices) - train_count)

            train_indices.extend(label_indices[:train_count])
            test_indices.extend(label_indices[train_count : train_count + test_count])

    else:
        indices = np.arange(len(df))
        np.random.shuffle(indices)

        train_indices = indices[:num_train]
        test_indices = indices[num_train : num_train + num_test]

    X_train = df.iloc[train_indices, :-1].to_numpy()
    y_train = df.iloc[train_indices, -1].to_numpy()

    X_test = df.iloc[test_indices, :-1].to_numpy()
    y_test = df.iloc[test_indices, -1].to_numpy()

    return X_train, y_train, X_test, y_test


# Load the dataset and split into train and test sets
DATA_FILEPATH = "data/checherboard_paper.csv"
X, Y, x_test, y_test = train_test_split(
    DATA_FILEPATH, num_train=50, num_test=50
)
# The proportion of train and test datapoints can be adjusted as needed.
# Doing some quick checks.
print("training_data: ", X.shape)
print("training_labels: ", Y.shape)
print("test_data: ", x_test.shape)
print("test_labels: ", len(y_test))

training_data:  (50, 2)
training_labels:  (50,)
test_data:  (49, 2)
test_labels:  49


In [126]:
type(X[0][0])

numpy.float64

In [127]:

circuit_executions = 0
def layer(x, params, wires, i0=0, inc=1):
    """Building block of the embedding ansatz"""
    i = i0
    for j, wire in enumerate(wires):
        qml.Hadamard(wires=[wire])
        qml.RZ(x[i % len(x)], wires=[wire])
        i += inc
        qml.RY(params[0, j], wires=[wire])

    qml.broadcast(unitary=qml.CRZ, pattern="ring", wires=wires, parameters=params[1])

def ansatz(x, params, wires):
    """The embedding ansatz"""
    for j, layer_params in enumerate(params):
        layer(x, layer_params, wires, i0=j * len(wires))


adjoint_ansatz = qml.adjoint(ansatz)


def random_params(num_wires, num_layers):
    """Generate random variational parameters in the shape for the ansatz."""
    return np.random.uniform(0, 2 * np.pi, (num_layers, 2, num_wires), requires_grad=True)

dev = qml.device("default.qubit", wires=3, shots=None)
wires = dev.wires.tolist()

@qml.qnode(dev)
def kernel_circuit(x1, x2, params):
    global circuit_executions
    circuit_executions += 1
    ansatz(x1, params, wires=wires)
    adjoint_ansatz(x2, params, wires=wires)
    return qml.probs(wires=wires)

def kernel(x1, x2, params):
    return kernel_circuit(x1, x2, params)[0]

In [128]:
params = random_params(num_wires=3, num_layers=6
)

In [129]:
kernel_value = kernel(X[0], X[1], params)
print(f"The kernel value between the first and second datapoint is {kernel_value:.3f}")

The kernel value between the first and second datapoint is 0.482


In [130]:
classes = np.unique(Y)
n_clusters = 8

centroids = []
class_centroids = []
centroid_labels = []
for c in classes:
    class_data = X[np.where(Y == c)[0]]
    centroids.append(np.mean(class_data, axis=0))
    class_centroids.append([np.mean(cluster, axis=0) for cluster in np.array_split(class_data, n_clusters)])
    centroid_labels.extend([[c] * n_clusters])

In [131]:
def centroid_kernel_matrix(X, centroid, ckernel):
    
    kernel_matrix = []

    for i in range(len(X)):
        kernel_matrix.append(ckernel(centroid, X[i]))

    return np.array(kernel_matrix)

In [132]:

def centroid_target_alignment(X, Y, centroid, kernel, l = 0.1, assume_normalized_kernel=False, rescale_class_labels=True):
   
    Y = np.asarray(Y)
    K = centroid_kernel_matrix(X, centroid, kernel)
    numerator = l * np.sum(Y * K)  
    denominator = np.sqrt(np.sum(K**2) * np.sum(Y**2))

    TA = numerator / denominator

    return TA

In [133]:
def loss_kao(X, Y, centroid, kernel, params, lambda_kao = 0.01):
    TA = centroid_target_alignment(X, Y, centroid, kernel)
    r = lambda_kao * np.sum(params ** 2)
    return 1 - TA + r

In [134]:
def loss_co(X, Y, centroid, kernel, cl, lambda_kao = 0.01):
    TA = centroid_target_alignment(X, Y, centroid, kernel)
    r = np.sum(np.maximum(cl - 1, 0) - np.minimum(cl, 0))
    return 1 - TA + r

In [135]:
class1_centroids = class_centroids[0]
class2_centroids = class_centroids[1]

centroid1_labels = centroid_labels[0]
centroid2_labels = centroid_labels[1]

centroid1 = centroids[0]
centroid2 = centroids[1]

In [136]:
current_alignment = qml.kernels.target_alignment(
            X,
            Y,
            lambda x1, x2: kernel(x1, x2, params),
            assume_normalized_kernel=True,
        )
print(f"Alignment = {current_alignment:.3f}")

Alignment = 0.076


In [137]:
from sklearn.svm import SVC
trained_kernel = lambda x1, x2: kernel(x1, x2, params)
trained_kernel_matrix = lambda X1, X2: qml.kernels.kernel_matrix(X1, X2, trained_kernel)
svm_trained = SVC(kernel=trained_kernel_matrix).fit(X, Y) 

In [138]:
def accuracy(classifier, X, Y_target):
    return 1 - np.count_nonzero(classifier.predict(X) - Y_target) / len(Y_target)

accuracy_trained = accuracy(svm_trained, x_test, y_test)
print(f"The accuracy of a kernel with trained parameters is {accuracy_trained:.3f}")

The accuracy of a kernel with trained parameters is 0.571


In [139]:
kao_class = 1
kao_subclass = 1
opt = qml.AdamOptimizer()
opt_centroid = qml.GradientDescentOptimizer(0.2)

circuit_executions = 0
#params = random_params(num_wires=6, num_layers=6)

# Your provided arrays: centroids, class_centroids, and centroid_labels
n_classes = len(classes)

for i in range(1500):
    
    centroid_idx = kao_class - 1  # Index for the current class/centroid
    cost = lambda _params: -loss_kao(
            np.vstack(class_centroids),  # Access current class clusters
            np.concatenate(centroid_labels),  # Labels for the current class
            centroids[centroid_idx],        # Current centroid
            lambda x1, x2: kernel(x1, x2, params),
            _params
        )
    
    centroid_cost = lambda _centroid: -loss_co(
            class_centroids[centroid_idx],  # Access current class clusters
            centroid_labels[centroid_idx],  # Labels for the current class
            centroids[centroid_idx],        # Current centroid
            lambda x1, x2: kernel(x1, x2, params),
            _centroid
        )

    params = opt.step(cost, params)
    centroids[centroid_idx] = opt_centroid.step(centroid_cost, centroids[centroid_idx])

    for sub_centroid_idx in range(len(class_centroids[centroid_idx])):
        class_centroids[centroid_idx][sub_centroid_idx] = opt_centroid.step(centroid_cost, class_centroids[centroid_idx][sub_centroid_idx])
    kao_class = (kao_class % n_classes) + 1

In [140]:
current_alignment = qml.kernels.target_alignment(
            X,
            Y,
            lambda x1, x2: kernel(x1, x2, params),
            assume_normalized_kernel=True,
        )
print(f"Alignment = {current_alignment:.3f}")

Alignment = 0.093


In [141]:
from sklearn.svm import SVC
# First create a kernel with the trained parameter baked into it.
trained_kernel = lambda x1, x2: kernel(x1, x2, params)

# Second create a kernel matrix function using the trained kernel.
trained_kernel_matrix = lambda X1, X2: qml.kernels.kernel_matrix(X1, X2, trained_kernel)

# Note that SVC expects the kernel argument to be a kernel matrix function.
svm_trained = SVC(kernel=trained_kernel_matrix).fit(X, Y) 

In [142]:
def accuracy(classifier, X, Y_target):
    return 1 - np.count_nonzero(classifier.predict(X) - Y_target) / len(Y_target)

accuracy_trained = accuracy(svm_trained, X, Y)
print(f"The accuracy of a kernel with trained parameters is {accuracy_trained:.3f}")

The accuracy of a kernel with trained parameters is 0.740


In [143]:
accuracy_trained = accuracy(svm_trained, x_test, y_test)
print(f"The accuracy of a kernel with trained {accuracy_trained}")

The accuracy of a kernel with trained 0.5918367346938775


In [146]:
circuit_executions = 0

params = random_params(num_wires=3, num_layers=6)
opt = qml.GradientDescentOptimizer(0.2)

for i in range(500):
    # Choose subset of datapoints to compute the KTA on.
    print(i)
    subset = np.random.choice(list(range(len(X))), 8)
    # Define the cost function for optimization
    cost = lambda _params: -qml.kernels.target_alignment(
        X[subset],
        Y[subset],
        lambda x1, x2: kernel(x1, x2, _params),
        assume_normalized_kernel=True,
    )
    # Optimization step
    params = opt.step(cost, params)

    # Report the alignment on the full dataset every 50 steps.
print(f"Circuit Executions: {circuit_executions}")
current_alignment = qml.kernels.target_alignment(
            X,
            Y,
            lambda x1, x2: kernel(x1, x2, params),
            assume_normalized_kernel=True,
        )
print(f"Alignment = {current_alignment:.3f}")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [147]:
from sklearn.svm import SVC
# First create a kernel with the trained parameter baked into it.
trained_kernel = lambda x1, x2: kernel(x1, x2, params)

# Second create a kernel matrix function using the trained kernel.
trained_kernel_matrix = lambda X1, X2: qml.kernels.kernel_matrix(X1, X2, trained_kernel)

# Note that SVC expects the kernel argument to be a kernel matrix function.
svm_trained = SVC(kernel=trained_kernel_matrix).fit(X, Y)

In [148]:
accuracy_trained = accuracy(svm_trained, X, Y)
print(f"The accuracy of a kernel with trained parameters is {accuracy_trained:.3f}")

The accuracy of a kernel with trained parameters is 0.840


In [149]:
accuracy_trained = accuracy(svm_trained, x_test, y_test)
print(f"The accuracy of a kernel with trained parameters is {accuracy_trained:.3f}")

The accuracy of a kernel with trained parameters is 0.633


In [1]:
from pxr import Usd, UsdGeom, Gf

# Create a new USD stage (empty .usd file)
stage = Usd.Stage.CreateNew("test_scene.usda")

# Define a root Xform (transform) for the stage
xform = UsdGeom.Xform.Define(stage, "/World")
xform.AddTranslateOp().Set(Gf.Vec3d(0, 0, 0))

# Create a sphere under the root Xform
sphere = UsdGeom.Sphere.Define(stage, "/World/Sphere")
sphere.GetRadiusAttr().Set(1.0)

# Create a cube under the root Xform
cube = UsdGeom.Cube.Define(stage, "/World/Cube")
cube.GetSizeAttr().Set(2.0)
cube.AddTranslateOp().Set(Gf.Vec3d(3, 0, 0))

# Save the stage
stage.GetRootLayer().Save()

print("USD scene created successfully!")

USD scene created successfully!
